<center>

<img src="https://kajabi-storefronts-production.kajabi-cdn.com/kajabi-storefronts-production/file-uploads/sites/2148158644/images/85d1d5-f44-11de-03a0-b8b70657ae6f_pinecone.jpeg" width=80%>
</center>

**[Pinecone](https://www.pinecone.io/)** is a fully managed vector database built specifically for **high-performance vector search**. It helps you **store, index, and search vector embeddings** at scale—without managing the infrastructure yourself.

To use **Pinecone** for semantic vector storage and search, we’ll need:

* `langchain`: chaining and embedding integration

* `pinecone-client`: interact with **Pinecone DB**

* Either:
    * `openai` + `tiktoken`: for **OpenAI embeddings** or
    * `google-generativeai`: for **Gemini embeddings**

In [6]:
# Install necessary libraries (using gemini embeddings)
!pip install langchain pinecone-client pypdf -q
!pip install "google-ai-generativelanguage>=0.6.18,<0.7.0" langchain-google-genai -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [11]:
loader = PyPDFLoader("../assets/MachineTranslationwithAttention.pdf")
loader

In [14]:
data = loader.load()
data.page_content

AttributeError: 'list' object has no attribute 'page_content'

In [5]:
print(data[1].page_content)

Neural Machine Translation with Attention
Mohammad Wasil Saleem
Matrikel-Nr.: 805779
Universit¨at Potsdam
saleem1@uni-potsdam.de
Sandeep Uprety
Matrikel-Nr. 804982
Universit¨at Potsdam
uprety@uni-potsdam.de
Abstract
In recent years, the success achieved
through neural machine translation has
made it mainstream in machine translation
systems. In this work, encoder-decoder
with attention system based on ”Neural
Machine Translation by Jointly Learning
to Align and Translate” by Bahdanau et al.
(2014) has been used to accomplish the
Machine Translation between English and
Spanish Language which has not seen
much research work done as compared
to other languages such as German and
French. We aim to demonstrate the re-
sults similar to the breakthrough paper on
which our work is based on. We achieved
a BLEU score of 25.37, which was close
enough to what Bahdanau et al. (2014)
achieved in their work.
1 Introduction
Machine Translation (MT) is the task of translat-
ing text without human assis

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20
)

In [8]:
text_chunks = text_splitter.split_documents(data)
text_chunks

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-11-05T20:59:50+00:00', 'author': '', 'keywords': '', 'moddate': '2021-11-05T20:59:50+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'rgid': 'PB:355917108_AS:1086893412356097@1636146991241', 'source': '../assets/MachineTranslationwithAttention.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/355917108\nNeural Machine Translation with Attention\nTechnical Report · August 2021\nDOI: 10.13140/RG.2.2.29381.37607/1\nCITATION\n1\nREADS\n5,448\n2 authors:\nMohammad Wasil Saleem\nUniversität Potsdam\n4 PUBLICATIONS\xa0\xa0\xa02 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nSandeep Uprety\nUniversität Potsdam\n1 PUBLICATION\xa0\xa0\xa01 CITATION\xa0\xa0\xa0